In [1]:
SKUs = '../Data/productSKUs_Nov2017_do_not_distribute.csv'
locations = '../Data/locations_table_Nov2017_Do_Not_Distribute.csv'
geography = '../Data/locations_zips_counties_regions3.csv'
geography_cleaned = '../Data/geography.csv'
sku_shard = '../Data/sku_shard.csv'
sku_shard_2 = '../Data/sku_shard_2.csv'
sku_shard_3 = '../Data/sku_shard_3.csv'
quarantine_csv = '../Data/quarantine.csv'
repaired = '../Data/repaired.csv'
transactions = '../Data/transactions.csv'
fragments = '../Data/fragments.csv'
knit_skus = '../Data/knit_skus.csv'
# knit_skus_groomed = '../Data/knit_skus_groomed.csv'
# scraps = '../Data/scraps.csv'

In [2]:
def grab_features(file, delimiter, 
                  record_number=1,
                  print_cols=False, 
                  print_record=False, 
                  print_features=False
                 ):
    """
    INPUT: CSV data file with headers
    OUTPUT: Prints (optionally) 
       * column names, formatted to copy into SQL CREATE TABLE command
       * sample record (values == types for first record) 
       * features (col -- val for first record)
    """
    with open(file, 'r') as f:
        cols = f.readline().split(delimiter)
        for _ in range(record_number-1):
            f.readline()
        record_1 = f.readline().split(delimiter)
        if print_cols:
            for col in cols:
                print col + ' VARCHAR,'
        if print_record:
            for datum in record_1:
                print datum + ' == ' + str(type(datum))
        if print_features:
            for col, v in zip(cols, record_1):
                print col + ' -- ' + v

In [3]:
def inspect_line(source_file, line_num, delimiter='\t', quarantine=False, q_file=None): 
    with open(source_file, 'r') as f:
        for i, line in enumerate(f):
            if i == line_num:
                vals = line.split(delimiter)
                print("Number of values: {}\n==========\n").format(len(vals))
                print("Raw text:\n%r\n==========\n") % line
                for i, val in enumerate(vals):
                    print "{}: {}".format(i, val)
                
                if quarantine:
                    with open(q_file, 'w') as f2:
                        f2.write(line)
            
            elif i > line_num:
                f.close()
                break

In [15]:
grab_features(transactions, '\t', record_number=21, print_features=True)

id -- 673315
wa_seller_org_id -- 757
seller_organization_id -- 523
wa_inventory_id -- 6034295410008482
wa_transaction_id -- 68981023
wa_inventory_parent_id -- 6034295410007770
wa_seller_location_id -- 593
seller_location_id -- 653
first_retail_sale_at -- 2017-01-06 02:20:02+00
most_recent_retail_sale_at -- 2017-07-24 16:56:47+00
wa_retail_location_id -- 868
retail_location_id -- 965
lowest_retail_unit_price -- 12.33
highest_retail_unit_price -- 17.16
avg_retail_unit_price -- 16.68
unit_weight -- 
generic_strain_id -- 
strain_display_name -- mixed
user_strain_name -- Mixed
strain_type_id -- 
is_prerolled -- f
user_product_description -- Indica Amazeballs
total_thc_amount -- 58.73
total_cbd_amount -- 1.38
total_potency_amount -- 60.11
thc_potency_range_id -- 9
cbd_potency_range_id -- 2
total_potency_range_id -- 9
cannabinoid_type_id -- 1
unit_weight_range_id -- 
retail_unit_price_range_id -- 7
inventory_type_id -- 30
retailer_name -- PAPER AND LEAF
generic_sku -- [Marijuana Mix Infused m

In [66]:
inspect_line(geography_cleaned, 0, delimiter=',')

Number of values: 64

Raw text:
'orgid,orgid_dupl,locationid,Unnamed,name,address1,address2,city,state,zip,deleted,locationtype,licensenum,id,locationexp,locationissue,status,districtcode,loclatitude,loclongitude,mailaddress1,mailaddress2,mailcity,mailstate,mailzip,locubi,producer,processor,retail,transactionid,transactionid_original,fifteenday_end,delete_time,code,county,pop2014,pop2010,sqmi,km2,Zip Code,City,County,ZIP Code Tabulation Area,Total Population, 2010,Population Density (Persons / Square Mile), 2010,Total Housing Units, 2010,Occupied Housing Units, 2010,Vacant Housing Units, 2010,Occupancy Rate (%), 2010,Vacancy Rate (%), 2010,Land Area (Square Miles), 2010,Total Area (Square Miles), 2010,Water Area (%), 2010,Region\r\n'

0: orgid
1: orgid_dupl
2: locationid
3: Unnamed
4: name
5: address1
6: address2
7: city
8: state
9: zip
10: deleted
11: locationtype
12: licensenum
13: id
14: locationexp
15: locationissue
16: status
17: districtcode
18: loclatitude
19: loclongitude
20: m

In [ ]:
def find_fragments(file, record_len=46, delimiter='\t'):
    count = 0
    with open(file, 'r') as f:
        for i, line in enumerate(f):
            vals = line.split(delimiter)
            if len(vals) != record_len:
                print "Line number: {} Values: {}".format(i, len(vals))
                count += 1
    print("Total fragments: {}".format(count))
    
def write_shard(read_file, write_file, num_records=100):
    f1 = open(read_file, 'r')
    f2 = open(write_file, 'w')
    for i, line in enumerate(f1):
        if i <= num_records + 1:
            f2.write(line)
        else: break
    f1.close()
    f2.close()

In [ ]:
def close_quotes_in_prod_descr(source, write_f, fragment_f):    
    counter = 0
    fragments = 0
    f1 = open(source, 'r')
    f2 = open(write_f, 'w')
    f3 = open(fragment_f, 'w')
    for line in f1:
        parsed = line.split('\t')
        if len(parsed) != 46:
            f3.write(line)
            fragments += 1
            continue
        else:
            prod_desc = parsed[21]

        if prod_desc.count('"') % 2 == 0:
            f2.write(line)
        else:
            fixed = prod_desc + '"'
            parsed[21] = fixed
            patched = '\t'.join(parsed)
            f2.write(patched)
            counter += 1
    
    f1.close()
    f2.close()
    f3.close()
    print(
    """\nProcessing complete
    Records repaired: {}
    Fragments written: {}"""
    ).format(counter, fragments)

In [ ]:
def knit_fragments(read_f, write_f, delimiter='\t'):
    """Takes in a csv file of sequential record fragments, fuses them and writes
    complete records to file.
    """
    f1 = open(read_f, 'r')
    f2 = open(write_f, 'w')
    counter = 0
    frag_22s = []
    frag_25s = []
    frag_19s = []
    frag_17s = []
    frag_12s = []
    for line in f1: # sort fragments into lists by frag-length
        if len(line.strip().split(delimiter)) == 22:
            frag_22s.append(line.strip())
        if len(line.strip().split(delimiter)) == 25:
            frag_25s.append(line.strip())
        if len(line.strip().split(delimiter)) == 19:
            frag_19s.append(line.strip())
        if len(line.strip().split(delimiter)) == 17:
            frag_17s.append(line.strip())
        if len(line.strip().split(delimiter)) == 12:
            frag_12s.append(line.strip())
            
    doubles = zip(frag_22s, frag_25s)
    triples = zip(frag_19s, frag_17s, frag_12s)
    
    for tup in doubles:
        f2.write(''.join(tup) + '\n')
        counter += 1
    for tup in triples:
        f2.write(''.join(tup) + '\n')
        counter += 1
    
    f1.close()
    f2.close()
    print("Total records recovered and written: {}").format(counter)

In [ ]:
specimen_cols = ',Unnamed: 0,orgid,locationid,name,address1,address2,city,state,zip,deleted,locationtype,licensenum,id,locationexp,locationissue,status,districtcode,loclatitude,loclongitude,mailaddress1,mailaddress2,mailcity,mailstate,mailzip,locubi,producer,processor,retail,transactionid,transactionid_original,fifteenday_end,delete_time,code,county,pop2014,pop2010,sqmi,km2,Zip Code,City,County,ZIP Code Tabulation Area,"Total Population, 2010","Population Density (Persons / Square Mile), 2010","Total Housing Units, 2010","Occupied Housing Units, 2010","Vacant Housing Units, 2010","Occupancy Rate (%), 2010","Vacancy Rate (%), 2010","Land Area (Square Miles), 2010","Total Area (Square Miles), 2010","Water Area (%), 2010",Region\r\n'
specimen = '260,260,196,1,SATURN GROUP,13215 SE 30TH ST,,bellevue,WA,98005,0,6,412112,400,1509519599,1497941999,ACTIVE (ISSUED),7J,47.583253,-122.163329,13215 SE 30TH ST,,BELLEVUE,WA,980054402,603344149,1,1,,101005715,304712,1411114512,,Code,King,"136,426","122,363",33.48,86.7,98005,Bellevue,King,98005,"17,714","2,361.29","8,070","7,539",531,93.42,6.58,7.5,7.51,0.09,405 Corridor\r\n'


In [ ]:
# segment = ',Unnamed: 0,orgid,locationid,'
# revised_seg = 'orgid, orgid_dupl, locationid, Unnamed,'

def remove_quotes(read_f, write_f):
    # Remove double-quotes from compound fields in source csv and writes lines to new csv.
    segment = ',Unnamed: 0,orgid,locationid,'
    revised_seg = 'orgid,orgid_dupl,locationid,Unnamed,'
    f1 = open(read_f ,'r')
    f2 = open(write_f, 'w')
    counter = 0
    for line in f1:
        cleaned = line.replace('"', '').replace(segment, revised_seg)#.rstrip()
        f2.write(cleaned)
        counter += 1
    print('Processing complete. {} lines written to file.').format(counter)
        
        

In [ ]:
remove_quotes(geography, geography_cleaned)

In [ ]:
grab_features(geography_cleaned, ',', record_number=5, print_features=True)

In [ ]:
iterator = 0

In [ ]:
line3 = '39,39,226,1,GAIA INDUSTRIES,17022 FROHNING RD,,monroe,WA,98272,0,5,412232,175,1525157999,1493967599,ACTIVE (ISSUED),7G,47.823599,-122.002588,1429 AVENUE D # 225,,SNOHOMISH,WA,982901742,603297007,1,1,,91567331,121565,1401880503,,Code,Snohomish,17,899,17,304,6.05,15.7,98272,Monroe,Snohomish,98272,27,942,255.69,9,398,8,822,576,93.87,6.13,109.28,110.05,0.7,405 Corridor\n'
line2 = '59,59,210,1,COPPERHEAD FARM,105 MILL ST,,prescott,WA,99348,0,5,412078,196,1504249199,1472885999,ACTIVE (ISSUED),7O,46.2986019,-118.308488,103 MILL ST,,PRESCOTT,WA,993489762,603324540,1,1,0,61598569,121606,1403694900,,Code,Walla Walla,309,318,0.4,1,99348,Prescott,Walla Walla,99348,1,364,2.63,512,441,71,86.13,13.87,519.6,527.42,1.48,405 Corridor\n'
line200 = '1310,1310,1315,1,WEST CHOICE,2805 6TH AVE,,tacoma,WA,98406,1,8,409509,1706,1488355199,1456214399,CLOSED (PERMANENT),7V,47.25568,-122.472924,PO BOX 2766,,GIG HARBOR,WA,983354766,603479772,0,0,1,61598569,31019537,1457585999,1467879311,First,Pierce,205,159,198,397,49.73,128.8,98406,Tacoma,Pierce,98406,21,610,5,063.61,10,220,9,562,658,93.56,6.44,4.27,4.67,8.71,Greater Tacoma\n'

In [ ]:
lst_3 = line3.split(',')
lst_2 = line2.split(',')
lst_200 = line200.split(',')

In [ ]:
len(lst_200)

In [45]:
line_lengths = []
with open(geography_cleaned, 'r') as f:
    for l in f:
        line_lengths.append(len(l.split(',')))
set(line_lengths)

{54, 55, 56, 57, 58, 59, 60, 61, 62, 64}

In [46]:
set_len = set(line_lengths)
for length in sorted(list(set_len)):
    print"{}: {}".format(length, line_lengths.count(length))

54: 57
55: 88
56: 83
57: 150
58: 38
59: 738
60: 772
61: 184
62: 5
64: 1


In [30]:
def print_first_line_of_length(read_f, line_len):  
    with open(read_f, 'r') as f:
        for i, spec in enumerate(f):
            if len(spec.split(',')) == line_len:
                print('Line number: {}\nRaw Text:\n{}').format(i, spec)
                f.close()
                break

In [47]:
print_first_line_of_length(geography_cleaned, 62)

Line number: 55
Raw Text:
589,589,905,1,LOCAL ROOTS MARIJUANA,212 W WINESAP RD STE 101, 102,,bothell,WA,98012,0,10,415222,738,1509519599,1499497199,ACTIVE (ISSUED),3H,47.8235207,-122.2357949,212 W WINESAP RD STE 101,,BOTHELL,WA,980120000,603441961,,,1,105024514,2905914,1428826506,,Code,King,36,567,33,505,12.1,31.3,98012,Bothell,Snohomish,98012,51,136,3,337.63,20,580,19,448,1,132,94.5,5.5,15.32,15.34,0.1,405 Corridor



In [ ]:
lens = []
for l in lengths:
    lens.append(l)
    
sorted(lens), len(lens)

In [ ]:
iterator += 1
iterator + 1, inspect_line(fragments, iterator)

In [ ]:
write_shard(SKUs, sku_shard_3, num_records=77001)

In [ ]:
print_line(transactions, 2)#, quarantine=True, q_file=quarantine_csv) 
# Line of bum record: 77983, 125
# One that works with apostrophe: 73588

# Bad file:
'758403\t968\t685\t6033539070000652\t29568233\t6033490610000803\t856\t847\t2016-04-14 16:33:13+00\t2016-05-04 17:19:45+00\t408\t440\t21.99\t54.98\t26.24\t7.00\t116\tSFV OG\tSFV OG\t3\tf\t"7gx50 sfv \'"smalls: 0807.  22.11/.04   cannarex"\t22.11\t0.04\t22.15\t6\t\t6\t1\t2\t10\t24\tCANNAREX\t[Usable Marijuana SFV OG][7.0g]\t[CANNAREX][Usable Marijuana SFV OG][7.0g]\t1313.34\t350.00\t50.00\t612.50\t12.25\t29528934\t2016-04-09 00:00:00+00\t50.00\t440\t408\t1105398\n'

In [ ]:
tf # pre_rolled
"7gx50 sfv \'"smalls: 0807. 22.11/.04 cannarex"" # product description
22.11 # total_thc_amount

In [ ]:
close_quotes_in_prod_descr(SKUs, transactions, fragments)

In [ ]:
with open(quarantine_csv, 'r') as f:
    contents = f.read()
contents

In [ ]:
q = '673421\t1309\t969\t6034748420019119\t107535332\t6034748420017464\t1170\t1181\t2017-07-19 23:11:40+00\t2017-07-31 23:01:36+00\t250\t205\t26.88\t26.88\t26.88\t\t1\tBlue Dream\tBlue Dream\t3\tf\t"PH Blue Dream 7464 .5g Clarity6033453330010001\n'

In [ ]:
len(q.split('\t'))

In [ ]:
3043190 - 3032996